### [Observations :]

* The students are more likely to perform (>70% in math and reading) in charter schools (>90% vs 53% for distict schools) from small to medium size (<2000). It is confirmed by the top 5 performing schools that are charter schools and the bottom 5 performing schools that are distict schools.

* The budgets are inversely related with the performances. The schools that have a smaller budget per student are perfoming better than the schools with a higher budget per student.

* The performances per school are stable for each grade level.

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

#usefull filtered data

math_pass = school_data_complete[school_data_complete["math_score"]>=70]
reading_pass = school_data_complete[school_data_complete["reading_score"]>=70]
overall_pass = school_data_complete[(school_data_complete["reading_score"] >= 70) &
                                    (school_data_complete["math_score"]>= 70)]

In [2]:
#wite a function to calculation multiple operations on multiple columns
f = {"School ID" : "nunique", "Student ID" : "count",
     "reading_score" : "mean", "math_score" : "mean"}
summary_df = pd.DataFrame(school_data_complete.agg(f))
summary_df = summary_df.transpose()
#other calculations
summary_df["Total Budget"] = school_data_complete["budget"].unique().sum()
summary_df["% Passing Math"] = (math_pass["student_name"].count())/summary_df["Student ID"]
summary_df["% Passing Reading"] = (reading_pass["student_name"].count())/summary_df["Student ID"]
summary_df["% Overall Passing"] = (overall_pass["student_name"].count())/summary_df["Student ID"]
#organise and format the data frame
summary_df = summary_df[["School ID","Student ID", "Total Budget",
                        "math_score","reading_score","% Passing Math",
                        "% Passing Reading","% Overall Passing"]]
summary_df[["School ID", "Student ID"]] = summary_df[["School ID", "Student ID"]].astype("int32")
summary_df = summary_df.style.format({"Total Budget": "${:,}",
                    "reading_score": "{:.2f}%","math_score": "{:.2f}%",
                    "% Passing Math": "{:.2%}","% Passing Reading" : "{:.2%}",
                    "% Overall Passing" : "{:.2%}" })
summary_df

,School ID,Student ID,Total Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428",78.99%,81.88%,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#create a grouped data frame
grouped_df = school_data_complete.groupby(["school_name", "type", "budget"])

#create a function to apply on the grouped data frame
f = {"student_name" : "count", "math_score" : "mean", "reading_score" : "mean"}
summary2_df = grouped_df.agg(f)
summary2_df = summary2_df.reset_index()
summary2_df["Per Student Budget"] = (summary2_df["budget"])/(summary2_df["student_name"])
summary2_df = summary2_df.rename( {"math_score" : "Average Math Score", 
                                   "reading_score" :"Average Reading Score", 
                                   'type': "School Type", 'student_name': "Total Students",
                                   'budget' : "Total School Budget"}, axis = 1)
#create athe grouped data for the count of student with a pass score in math and reading
a = math_pass.groupby(["school_name"])["math_score"].count()
b = reading_pass.groupby(["school_name"])["math_score"].count()
c = overall_pass.groupby(["school_name"])["math_score"].count()

#calculation for passing % and create a data frame to merge
math = pd.DataFrame(a/list(summary2_df["Total Students"]))
reading = pd.DataFrame(b/list(summary2_df["Total Students"]))
overall = pd.DataFrame(c/list(summary2_df["Total Students"]))
#merge the 3 dataframes for the passing scores
summary3 = pd.merge(pd.merge(math, reading, on = "school_name"), overall, on = "school_name")
#merge the 2 dataframes to create the final data frame
summary_final_df = pd.merge(summary2_df, summary3, on = "school_name")
#formatting
summary_final_df = summary_final_df[['school_name', 'School Type', 'Total Students','Total School Budget', 
                                     'Per Student Budget', 'Average Math Score','Average Reading Score', 
                                     'math_score_x', 'math_score_y','math_score']]
summary_final_df = summary_final_df.rename({'school_name' : "School Names",
                                            'math_score_x' : "% Passing Math", 
                                            'math_score_y' :"% Passing Reading",
                                            'math_score': "% Overall Passing"}, axis = 1)
#store the dataframe unstyled summary_final_df for reuse
summary_final1_df = summary_final_df
summary_final1_df = summary_final1_df.style.format({'Total School Budget' : "${:,}",
                                                  'Per Student Budget': "${:,.2f}",
                                                  'Average Math Score' : "{:.2f}%", 
                                                  'Average Reading Score' : "{:.2f}%",
                                                  '% Passing Math' : "{:.2%}", 
                                                  '% Passing Reading' : "{:.2%}",
                                                  '% Overall Passing' : "{:.2%}"})

summary_final1_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928",$628.00,77.05%,81.03%,66.68%,81.93%,54.64%
1,Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.06%,83.98%,94.13%,97.04%,91.33%
2,Figueroa High School,District,2949,"$1,884,411",$639.00,76.71%,81.16%,65.99%,80.74%,53.20%
3,Ford High School,District,2739,"$1,763,916",$644.00,77.10%,80.75%,68.31%,79.30%,54.29%
4,Griffin High School,Charter,1468,"$917,500",$625.00,83.35%,83.82%,93.39%,97.14%,90.60%
5,Hernandez High School,District,4635,"$3,022,020",$652.00,77.29%,80.93%,66.75%,80.86%,53.53%
6,Holden High School,Charter,427,"$248,087",$581.00,83.80%,83.81%,92.51%,96.25%,89.23%
7,Huang High School,District,2917,"$1,910,635",$655.00,76.63%,81.18%,65.68%,81.32%,53.51%
8,Johnson High School,District,4761,"$3,094,650",$650.00,77.07%,80.97%,66.06%,81.22%,53.54%
9,Pena High School,Charter,962,"$585,858",$609.00,83.84%,84.04%,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
summary_final2_df = summary_final_df

summary_final2_df = summary_final2_df.sort_values( '% Overall Passing',ascending = False).iloc[:5]

summary_final2_df = summary_final2_df.style.format({'Total School Budget' : "${:,}",
                                                  'Per Student Budget': "${:,.2f}",
                                                  'Average Math Score' : "{:.2f}%", 
                                                  'Average Reading Score' : "{:.2f}%",
                                                  '% Passing Math' : "{:.2%}", 
                                                  '% Passing Reading' : "{:.2%}",
                                                  '% Overall Passing' : "{:.2%}"})
summary_final2_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.06%,83.98%,94.13%,97.04%,91.33%
12,Thomas High School,Charter,1635,"$1,043,130",$638.00,83.42%,83.85%,93.27%,97.31%,90.95%
4,Griffin High School,Charter,1468,"$917,500",$625.00,83.35%,83.82%,93.39%,97.14%,90.60%
13,Wilson High School,Charter,2283,"$1,319,574",$578.00,83.27%,83.99%,93.87%,96.54%,90.58%
9,Pena High School,Charter,962,"$585,858",$609.00,83.84%,84.04%,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
summary_final3_df = summary_final_df

summary_final3_df = summary_final3_df.sort_values( '% Overall Passing').iloc[:5]

summary_final3_df = summary_final3_df.style.format({'Total School Budget' : "${:,}",
                                                  'Per Student Budget': "${:,.2f}",
                                                  'Average Math Score' : "{:.2f}%", 
                                                  'Average Reading Score' : "{:.2f}%",
                                                  '% Passing Math' : "{:.2%}", 
                                                  '% Passing Reading' : "{:.2%}",
                                                  '% Overall Passing' : "{:.2%}"})
summary_final3_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,3999,"$2,547,363",$637.00,76.84%,80.74%,66.37%,80.22%,52.99%
2,Figueroa High School,District,2949,"$1,884,411",$639.00,76.71%,81.16%,65.99%,80.74%,53.20%
7,Huang High School,District,2917,"$1,910,635",$655.00,76.63%,81.18%,65.68%,81.32%,53.51%
5,Hernandez High School,District,4635,"$3,022,020",$652.00,77.29%,80.93%,66.75%,80.86%,53.53%
8,Johnson High School,District,4761,"$3,094,650",$650.00,77.07%,80.97%,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
summary4_df = pd.DataFrame()
#groupbygrade  
summary4_df["9th"] = school_data_complete[(school_data_complete["grade"] == "9th")].groupby("school_name")["math_score"].mean()
summary4_df["10th"] = school_data_complete[school_data_complete["grade"] == "10th"].groupby("school_name")["math_score"].mean()
summary4_df["11th"] = school_data_complete[school_data_complete["grade"] == "11th"].groupby("school_name")["math_score"].mean()
summary4_df["12th"] = school_data_complete[school_data_complete["grade"] == "12th"].groupby("school_name")["math_score"].mean()
#formatting
summary4_df = summary4_df.style.format({"9th" : "{:.2f}%",
                                        "10th" : "{:.2f}%", 
                                        "11th" : "{:.2f}%",
                                        "12th" : "{:.2f}%"})
summary4_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
summary5_df = pd.DataFrame()
#groupbygrade = 
summary5_df["9th"] = school_data_complete[(school_data_complete["grade"] == "9th")].groupby("school_name")["reading_score"].mean()
summary5_df["10th"] = school_data_complete[school_data_complete["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
summary5_df["11th"] = school_data_complete[school_data_complete["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
summary5_df["12th"] = school_data_complete[school_data_complete["grade"] == "12th"].groupby("school_name")["reading_score"].mean()
#formatting
summary5_df = summary5_df.style.format({"9th" : "{:.2f}%",
                                        "10th" : "{:.2f}%", 
                                        "11th" : "{:.2f}%",
                                        "12th" : "{:.2f}%"})
summary5_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
#reuse the school summary table
spending_summary = summary_final_df
bins = [0, 584, 630, 645, 676]
labels = ["<$584", "$585-629", "$630-644", "$645-675"]
spending_summary["Per Student Budget Range"] = pd.cut(spending_summary["Per Student Budget"],
                       bins = bins, labels = labels, right=False)
f = {"Average Math Score" : "mean","Average Reading Score" : "mean",
    "% Passing Math" : "mean","% Passing Reading": "mean","% Overall Passing" : "mean" }
spending_summary = spending_summary.groupby(["Per Student Budget Range"]).agg(f)

#formatting
spending_summary = spending_summary.rename({"math_score": "Average Math Score",
                                            "reading_score": "Average Reading Score"}, axis = 1)
spending_summary = spending_summary.style.format({'Average Math Score' : "{:.2f}%", 
                                                  'Average Reading Score' : "{:.2f}%",
                                                  '% Passing Math' : "{:.2%}", 
                                                  '% Passing Reading' : "{:.2%}",
                                                  '% Overall Passing' : "{:.2%}"}) 

spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Budget Range,,,,,
<$584,83.46%,83.93%,93.46%,96.61%,90.37%
$585-629,81.90%,83.16%,87.13%,92.72%,81.42%
$630-644,78.52%,81.62%,73.48%,84.39%,62.86%
$645-675,77.00%,81.03%,66.16%,81.13%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
#reuse the school summary table
summary_final_df["size"] = list(school_data_complete.groupby("school_name")["size"].unique())
size_summary = summary_final_df
bins = [0, 999, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
size_summary["size_range"] = pd.cut(size_summary["size"],
                       bins = bins, labels = labels, right=True)
#f is defined already to claculate the columns
size_summary = size_summary.groupby(["size_range"]).agg(f)
#formatting
size_summary = size_summary.rename({"math_score": "Average Math Score",
                                    "reading_score": "Average Reading Score"}, axis = 1)
size_summary = size_summary.style.format({'Average Math Score' : "{:.2f}%", 
                                          'Average Reading Score' : "{:.2f}%",
                                          '% Passing Math' : "{:.2%}", 
                                          '% Passing Reading' : "{:.2%}",
                                          '% Overall Passing' : "{:.2%}"})

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
size_range,,,,,
Small (<1000),83.82%,83.93%,93.55%,96.10%,89.88%
Medium (1000-2000),83.37%,83.86%,93.60%,96.79%,90.62%
Large (2000-5000),77.75%,81.34%,69.96%,82.77%,58.29%


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
#f is defined already to claculate the columns
type_summary = summary_final_df.groupby("School Type").agg(f)
#formatting
type_summary = type_summary.style.format({'Average Math Score' : "{:.2f}%",
                                          'Average Reading Score' : "{:.2f}%",
                                          '% Passing Math' : "{:.2%}", 
                                          '% Passing Reading' : "{:.2%}",
                                          '% Overall Passing' : "{:.2%}"})

type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47%,83.90%,93.62%,96.59%,90.43%
District,76.96%,80.97%,66.55%,80.80%,53.67%
